In [2]:
import pandas as pd
import os

In [3]:
predict_files=os.listdir('./submission')

In [4]:
predict_files

['submission_DenseNet121.csv',
 'submission_Resnet50.csv',
 '.ipynb_checkpoints',
 'submission_EfficientNetB0.csv']

In [9]:
DenseNet121_predict = pd.read_csv("submission/submission_DenseNet121.csv")
Resnet50_predict = pd.read_csv("submission/submission_Resnet50.csv")
EfficientNetB0_predict = pd.read_csv("submission/submission_EfficientNetB0.csv")

In [14]:
DenseNet121_predict = DenseNet121_predict.rename(columns={"Label": "D_predict"})
Resnet50_predict = Resnet50_predict.rename(columns={"Label": "R_predict"})
EfficientNetB0_predict = EfficientNetB0_predict.rename(columns={"Label": "E_predict"})

In [23]:
final_predict = DenseNet121_predict.merge(Resnet50_predict, on="ID").merge(EfficientNetB0_predict, on="ID")

In [25]:
import itertools
columns = ['D_predict', 'R_predict', 'E_predict']
for col1, col2 in itertools.combinations(columns, 2):
    diff_count = (final_predict[col1] != final_predict[col2]).sum()
    print(f"{col1} 和 {col2} 預測不同的個數: {diff_count}")

D_predict 和 R_predict 預測不同的個數: 195
D_predict 和 E_predict 預測不同的個數: 118
R_predict 和 E_predict 預測不同的個數: 205


In [26]:
from collections import Counter
def majority_vote(row):
    votes = [row["D_predict"], row["R_predict"], row["E_predict"]]
    return Counter(votes).most_common(1)[0][0]  

In [27]:
final_predict["VOTE_predict"] = final_predict.apply(majority_vote, axis=1)

In [28]:
final_predict.head(20)

,ID,D_predict,R_predict,E_predict,VOTE_predict
0,test_00000.png,1,1,1,1
1,test_00001.png,2,2,2,2
2,test_00002.png,5,5,5,5
3,test_00003.png,0,1,0,0
4,test_00004.png,2,2,2,2
5,test_00005.png,5,5,5,5
6,test_00006.png,5,5,5,5
7,test_00007.png,5,5,5,5
8,test_00008.png,0,0,0,0
9,test_00009.png,2,2,2,2


In [29]:
submission_VOTE=final_predict[['ID','VOTE_predict']]
submission_VOTE=submission_VOTE.rename(columns={"VOTE_predict": "Label"})
submission_VOTE.to_csv("submission/submission_VOTE.csv",index=False)